In [1]:
import requests
from datetime import datetime
from datetime import date
import pandas as pd
import os
from dotenv import load_dotenv

load_dotenv()

cities_list = pd.read_excel('cities_list.xlsx')

API_KEY = os.getenv('API_KEY')

def get_air_pollution_data(API_KEY, lat, lon):
    url = f'http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&appid={API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f'Error requisition: {response.status_code}')
        return None

In [2]:
cities_list

,name,lon,lat,country
0,Abidjan,-4.00167,5.35444,Ivory Coast
1,Abu Dhabi,54.39696,24.45118,United Arab Emirates
2,Abuja,7.49508,9.05785,Nigeria
3,Accra,-0.19690,5.55602,Ghana
4,Addis Ababa,38.74689,9.02497,Ethiopia
...,...,...,...,...
366,Yaounde,11.51667,3.86667,Cameroon
367,Yekaterinburg,60.61220,56.85190,Russia
368,Yerevan,44.51361,40.18111,Armenia
369,Yokohama,139.65000,35.43333,Japan


In [3]:
def process_pollution_data(data, city, country):
    if not data or 'list'not in data:
        return None
    
    formatted_data = []
    for item in data['list']:
        row = {
            'dt': item['dt'],
            'aqi': item['main']['aqi'],
            'city': city,
            'country': country,
            'co': item['components']['co'],
            'no': item['components']['no'],
            'no2': item['components']['no2'],
            'o3': item['components']['o3'],
            'so2': item['components']['so2'],
            'pm2_5': item['components']['pm2_5'],
            'pm10': item['components']['pm10'],
            'nh3': item['components']['nh3']

        }
        if isinstance(item.get('components'), dict):
            row.update(item['components'])
            if 'coord' in item:
                row['lon'] = item['coord'].get('lon', None)
                row['lat'] = item['coord'].get('lat', None)
        formatted_data.append(row)
        
    return formatted_data

In [22]:
def format_date(timestamp):
    if pd.isna(timestamp):
        return None
    elif isinstance(timestamp, (int, float)):
        return pd.to_datetime(timestamp, unit='s')
    return None

In [23]:
all_data = []
for index, row in cities_list.iterrows():
    lon = row['lon']
    lat = row['lat']
    country = row['country']
    city = row['name']
    data = get_air_pollution_data(API_KEY, lat, lon)
    
    if data:
        city_data = process_pollution_data(data, city, country)
        if city_data:
            all_data.append(city_data)

In [24]:
correction_data = [item for sublist in all_data for item in sublist if isinstance(item, dict)]
df = pd.DataFrame(correction_data)

In [30]:
df['dt'] = df['dt'].apply(format_date)

In [31]:
df

,dt,aqi,city,country,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,2025-06-05 01:00:00,1,Abidjan,Ivory Coast,102.12,0.0,0.13,37.51,0.10,1.31,2.23,0.00
1,2025-06-05 02:00:00,1,Abidjan,Ivory Coast,100.74,0.0,0.13,37.04,0.09,1.24,2.04,0.00
2,2025-06-05 03:00:00,1,Abidjan,Ivory Coast,99.63,0.0,0.13,36.57,0.09,1.20,1.90,0.00
3,2025-06-05 04:00:00,1,Abidjan,Ivory Coast,98.64,0.0,0.13,36.14,0.08,1.16,1.78,0.00
4,2025-06-05 05:00:00,1,Abidjan,Ivory Coast,97.76,0.0,0.15,35.64,0.09,1.16,1.74,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
35611,2025-06-08 20:00:00,2,Zagreb,Croatia,102.90,0.0,1.33,90.56,0.76,21.73,34.90,0.55
35612,2025-06-08 21:00:00,2,Zagreb,Croatia,104.57,0.0,1.36,88.33,0.70,21.15,33.25,0.63
35613,2025-06-08 22:00:00,2,Zagreb,Croatia,106.08,0.0,1.37,82.53,0.65,18.24,26.92,0.60
35614,2025-06-08 23:00:00,2,Zagreb,Croatia,117.24,0.0,1.38,77.38,0.57,14.53,20.13,0.47
